# **Scraping de titulares de varios periódicos**

## 1.Scrapeo de datos

In [ ]:
import urllib3
from copy import deepcopy
import json
from multiprocessing.pool import ThreadPool
import re

from bs4 import BeautifulSoup

In [ ]:
def get_request(uri):
    """Devuelve el request sobre una URL

    Args:
        uri (str): La url de donde obtener el Request

    Returns:
        Request: La request a la URL
    """
    ua = "Mozilla/5.0 (Linux; U; Android 2.2; en-us; Nexus One Build/FRF91) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1"  
    h = {"User-Agent": ua}
    httpPool = urllib3.PoolManager()
    return httpPool.request('GET',uri,fields=None,headers=h)

In [ ]:
claves = {}

def get_data(map):
    """Devuelve la Lista de Datos transformados para su utilización

    Args:
        Map (Map<Key,Value>): Mapa con el periódico y los valores

    Returns:
        List: Frase Str, Url Str, Peso Float
    """
    dato=map[1]
    datos=[]
    soup = BeautifulSoup(get_request(dato[1]).data,'html.parser')
    urls=[]
    for text in soup.find_all(dato[2],{"class":dato[3]} if len(dato[3])>0 else None):
        texto = text.text.replace("\n","")
        texto = texto.replace("\r","")
        texto = texto.replace("\t","")
        if dato[4] == ".":
            enlace = text
            bEnlace = text[dato[5]]
        else:
            enlace=text.findChildren(dato[4] , recursive=False)
            if len(enlace)==0:
                continue
            bEnlace = enlace[0][dato[5]]
        if bEnlace.startswith("/"):
            bEnlace=dato[1]+bEnlace
        datosenlace=[texto,bEnlace,0]
        datos.append(datosenlace)
        urls.append(bEnlace)
    claves[dato[0]]=urls
    return datos

In [ ]:


# Recogemos los datos del fichero .json 
with open('Media/medios.json',"r") as f:
    urls = json.load(f)


DATA=[]
# Guardamos los valores transformados en una lista con su key para trastear con ellos
for key,x in urls.items():
    DATA.append([key,x['url'],x['tag'],x['clase'],x['tagURL'],x['valueURL']])

# Creamos un ThreadPool donde se ejecutará en varios procesos la búsqueda de datos, sin esto en búsqueda de 10 páginas tardaría más de 10 segundos
pool = ThreadPool(len(DATA))
datos=[]
v = pool.map(get_data, enumerate(DATA)) 
datos = [item for lista in v for item in lista]

In [ ]:
# Vemos aquí los datos de cada página con toda su URL

datos

In [ ]:
import nltk
from nltk.corpus import stopwords

#nltk.download('stopwords')

stop_words = stopwords.words('spanish')

stop_words += ["\u200b", "\xa0", "para", "como", "puede","cómo", "hacer", "forma", "parte", "hace", "además", "según", "pueden", "ser","tras"]

stop_words[:10]

# Definimos los StopWords para eliminar en el contador 

In [ ]:
# Añadimos a un regex todas las palabras no permitidas
regex = "|".join(stop_words)

# Hacemos deepcopy de los datos que trabajamos luego en el conteo
frasesEditadas = deepcopy(datos) 
for index,dato in enumerate(datos):
    viejaFrase=deepcopy(dato[0])
    # Filtramos las palabras
    fraseNueva = re.sub(f"\\b({regex})\\b", " ", viejaFrase)
    frasecompleta = fraseNueva.strip ()
    palabras = frasecompleta.split ()
    # Juntamos la frase completa con filtros que eliminan caracteres no deseados
    frasecompleta = " ".join([re.sub(r'^\W|\W$', '', p.lower()) for p in palabras if len(re.sub(r'^\W|\W$', '', p)) > 3])
    frasesEditadas[index] = [frasecompleta,frasesEditadas[index][1],frasesEditadas[index][2]]

In [ ]:
# Vemos aquí las frases editadas

frasesEditadas

In [ ]:
frecuencias = {}

# Recorro en el conteo de frecuencia de palabras
textoEntero = " ".join([sublista[0] for sublista in frasesEditadas])
for palabra in textoEntero.split():
    palabra = palabra.strip() 
    if palabra in frecuencias:
        frecuencias[palabra] += 1 # Aumenta la frecuencia si la palabra ya existe
    else:
        frecuencias[palabra] = 1 # Crea una nueva entrada si la palabra no existe

# Y calculo el valor de cada frase editada 
for index,portadaE in enumerate(frasesEditadas):
    palabras = portadaE[0].split()
    valor=0
    for palabra in palabras:
      valor+=frecuencias[palabra]
    l = len(datos[index][0].split())
    frasesEditadas[index][2]=valor/(l)
    datos[index][2]=valor/l

frecuencias

In [ ]:
# Saco el valor relativo de cada una de las frases

tope=max(datos,key= lambda x:x[2])[2]
for index,i in enumerate(datos):
    datos[index][2]=(i[2]/tope)
datos

In [ ]:
titularesOrdenados=sorted(datos,key= lambda x:x[2],reverse=True)

# Borramos si coincide titular
dicc = {}
for x in titularesOrdenados:
    dicc[x[0]] = x

titularesOrdenados = list(dicc.values())

# 2. **Amosar un gráfico de barras coas 10 palabras que se repiten máis**

In [ ]:
import matplotlib.pyplot as plt

# Ordenamos el listado y pillamos los mejores 10
diccionario_ordenado = sorted(frecuencias.items(), key=lambda x: x[1], reverse=True)
diez_mas_grandes = dict(diccionario_ordenado[:10])

# Los ponemos en el plt.bar
plt.bar(diez_mas_grandes.keys(),diez_mas_grandes.values())
plt.xlabel("PALABRAS")
plt.xticks(rotation=-90)

# 3. **Escribir a un arquivo HTML: resultado-ano-mes-dia.html (ordeado polo peso, de modo que os titulares que aparecen máis grandes, aparezan ao inicio)**

**Librería para manejo de HTML**

In [ ]:
#!conda install -y -c conda-forge airium

In [ ]:
import time
from airium import Airium

a = Airium()


# Librería para el manejo de HTML
a('<!DOCTYPE html>')
with a.html(lang="pl"):
    with a.head():
        a.meta(charset="utf-8")
        a.title(_t="Página de noticias")
    with a.body():
        # Decimos que por cada titular ordenado ponemos el peso relativo +1, la URL a la noticia como enlace y el texto de la noticia
        for portada in titularesOrdenados:
            with a.p(style=f"font-size:{portada[2]+1}em;"):
                with a.a(href=portada[1]):
                    a(portada[0])

html = str(a)  # Casteo de los datos

# Escribimos el HTML con el año, mes y día
with open(f"HTML/resultado-{time.localtime().tm_year}-{time.localtime().tm_mon}-{time.localtime().tm_mday}.html","w+") as f:
    f.write(html)


# 4. **Amosar por pantalla os 20 titulares e a URL que teñan maior peso relativo (“relevancia”)**

In [ ]:
# Recorremos los titulares ordenados anteriormente y recogemos los 20 primeros
for i in titularesOrdenados[:20]:
    print(f"URL: {i[1]} - FRASE: {i[0]}")

# 5. **Suxire un método mellor para analizar a relevancia dos titulares**

Se puede utilizar el análisis de de sentimiento, esto implica utilizar técnicas de procesamiento de lenguaje natural para determinar si el titular es positivo, natural o neutro, esto implica que los positivos y negativos seán más relevantes, mientras que los neutros los sean menos, teniendo en cuenta también la repetición de palabras y su peso en las frases, por lo que nos daría una mayor precisión juntando los dos métodos

# 6. **Mete os titulares, medio, URL e peso relativo nunha BBDD de SQLite**

In [ ]:
#!conda install -y sqlite3 

In [ ]:
connection=None

In [ ]:
import sqlite3

# Conectamos siguiendo el patrón SingleTone
if not(connection):
    connection=sqlite3.connect("Media/titulares.db")

In [ ]:
# Creamos el cursor y si existe la tabla en la BBDD nos lo dice
cursor = connection.cursor()
try:
    cursor.execute("CREATE TABLE titulares (titular TEXT,medio TEXT, url TEXT, prelativo REAL)")
except:
    print("La tabla ya está creada")

In [ ]:
def buscador_titular_peso(URL):
    """Devuelve el titular y su peso relativo

    Args:
        URL (str): URL del titular

    Returns:
        List: Devuelve el titular y peso
    """
    for i in datos:
        if i[1] == URL:
            return i[0],i[2]

In [ ]:
valores = []

# Buscamos los datos a insertar y lo ejecutamos con executemany
for key,values in claves.items():
    for url in values:
        titular,peso=buscador_titular_peso(url)
        valores.append([titular,key,url,peso])


cursor.executemany("""
    INSERT INTO titulares ('titular', 'medio', 'url', 'prelativo')
    VALUES (?,?,?,?)""", valores)

# Si quisieramos insertar y que los cambios no queden pendientes
#connection.commit()

In [ ]:
# Vemos que se insertan o por lo menos están para commit

cursor.execute("SELECT * FROM titulares order by prelativo desc")

rows = cursor.fetchall()

rows

In [ ]:
# Y cerramos conexión

cursor.close()
connection.close()